# Initialization 

In [0]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from statistics import mean
from timeit import default_timer as timer
from math import sqrt, exp
from random import shuffle, random, randint
from google.colab import files

In [0]:
# Download the "Coors_140.csv" from the GitHub repository and upload it
uploaded = files.upload()

In [0]:
coors = pd.read_csv("Coors_140.csv") 
coords = coors.to_numpy()
coords = np.delete(coords, 0,1)
amount_of_coordinates = 40
Coordinates = coords[:amount_of_coordinates]

#Plotting
plt.scatter(Coordinates[0][1], Coordinates[0][2], c='r', marker='D')
plt.scatter(Coordinates[1:,1], Coordinates[1:,2])

In [0]:
# If you prefer to generate the coordinates randomly, follow this step.

nodes = 40
Coordinates = np.ones((nodes, 3))
Coordinates[0] = 1, 0.5, 0.5

for i in range(1, nodes):
  Coordinates[i] = int(i+1), random(), random()

#Plotting
plt.scatter(Coordinates[0][1], Coordinates[0][2], c='r', marker='D')
plt.scatter(Coordinates[1:,1], Coordinates[1:,2])

# Distance Matrix

In [0]:
def Distance(x1, y1, x2, y2):
    return sqrt((x1-x2)**2+(y1-y2)**2)

#Creating Distance Matrix
length = len(Coordinates)
Distance_Matrix = np.zeros((length, length))
for i in range(length):
  for j in range(length):
    Distance_Matrix[i][j] = Distance(Coordinates[i][1],  Coordinates[i][2], Coordinates[j][1], Coordinates[j][2])

# Functions

In [0]:
def First_Tour():
  FirstTour = list(range(2, length+1))
  shuffle(FirstTour)
  FirstTour = [1] + FirstTour + [1]
  return FirstTour

In [0]:
def iter_num(temp, rate, mitemp):
  number = 0 
  while True:
    temp *= rate
    if temp <= mitemp:break
    number+=1
  return number

In [0]:

def Plotting(tour, coords):
    n=len(tour)
    longitude=[]
    latitude=[]
    
    for i in range(n):
            longitude.append(coords[tour[i]-1, 1])
            latitude.append(coords[tour[i]-1, 2])
    plt.figure()    
    plt.plot([longitude[i] for i in range(n)],
              [latitude[i] for i in range(n)],'-.', color='cornflowerblue')
    plt.scatter(Coordinates[0][1], Coordinates[0][2], c = 'r', marker='D')
    plt.scatter(Coordinates[1:,1], Coordinates[1:,2])

In [0]:
def FindCurrentCost(CurTour):
    temporaryDis=0
    for i in range(len(CurTour)-1):    
       temporaryDis += Distance_Matrix[CurTour[i]-1,CurTour[i+1]-1]
    return temporaryDis 

# New Tour

In [0]:
def NewTourRev(seq):
    tmp=seq[0]
    del seq[0],seq[-1]

    i = randint(0,  len(seq)- 2)
    j = randint(i+2, len(seq))

    seq[i:j] = reversed(seq[i:j])
    
    seq = [tmp] + seq + [tmp]
    
    return seq

# Simulated Annealing

In [0]:
FirstTour = First_Tour()
Shortest_Tour_Cost=99999
Temperature = 20
alpha = 0.9975
mintemp = 0.001
innerloop = len(Coordinates)*4

Shortest_Tour_Graph=[]
Xo = FirstTour[:]
Fxo=FindCurrentCost(FirstTour)
start = timer()

while True:
    for i in range(innerloop):
        Xnew=NewTourRev(Xo[:])
        Fxnew=FindCurrentCost(Xnew)
        Df=Fxo-Fxnew
        if Df > 0:
            Xo=Xnew[:]
            Fxo = Fxnew    
        else:
            p = exp(Df/Temperature)
            ran=random()             
            if ran < p:
                Xo=Xnew[:]
                Fxo = Fxnew
 
        if Fxo < Shortest_Tour_Cost:
            Shortest_Tour_Cost=Fxo
            Shortest_Tour= Xo[:]

        Shortest_Tour_Graph.append(Shortest_Tour_Cost) 
    Temperature*= alpha
    if Temperature <= mintemp:break  

end = timer() 

print("Shortest Tour:",Shortest_Tour)
print("Shortest Tour cost:","{0:.3f}".format(round(Shortest_Tour_Cost,3)))
print("Calculation Time:","{0:.4f}".format(round((end - start),4)), "sec")
Plotting(Shortest_Tour, Coordinates)
plt.figure()
plt.plot(Shortest_Tour_Graph,'r')